# Filter Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import csv

import json
import unittest

from common import Printable, projdir

from entrant import Entrant

from constants import *

## Filter Class

Class to apply filters and cache results

In [2]:
class Filter(Printable):
    def __init__(self, entrants, verbosity=1):
        '''Initialise speed run object'''    

        super().__init__(verbosity=verbosity)
        
        self.entrants = entrants
        
        self.cache = {}


    def applyFilter(self, entrants, filterRule):
        '''Retrieve entrants that match a single filter criteria, without cache'''

        filteredEntrants = {}

        if '=' in filterRule:
            filterParts = filterRule.split('=')
            for entrantId in entrants:
                value = entrants[entrantId].getValue(filterParts[0])
                if '%' in filterParts[1]:
                    wildCardParts = filterParts[1].split('%')
                    if value.startswith(wildCardParts[0]) and value.endswith(wildCardParts[1]):
                        filteredEntrants[entrantId] = entrants[entrantId]
                elif value == filterParts[1]:
                    filteredEntrants[entrantId] = entrants[entrantId]
        elif '<>' in filterRule:
            filterParts = filterRule.split('<>')
            for entrantId in entrants:
                value = entrants[entrantId].getValue(filterParts[0])
                if '%' in filterParts[1]:
                    wildCardParts = filterParts[1].split('%')
                    if value and not (value.startswith(wildCardParts[0]) and value.endswith(wildCardParts[1])):
                        filteredEntrants[entrantId] = entrants[entrantId]
                elif value and value != filterParts[1]:
                    filteredEntrants[entrantId] = entrants[entrantId]
        else:
            raise ValueError('Invalid filter rule "{}"'.format(filterRule))

        return filteredEntrants


    def getEntrants(self, filterText):
        '''Retrieve entrants that match multiple filter criteria, utilising cache'''

        entrants = self.entrants

        filterRules = filterText.split(',')
        filterKey = ''
        
        for filterRule in filterRules:
            if filterKey:
                filterKey = filterKey + ',' + filterRule
            else:
                filterKey = filterRule
                
            if filterKey in self.cache:
                entrants = self.cache[cacheId]
            else:
                entrants = self.applyFilter(entrants, filterRule)
                self.cache[filterKey] = entrants

        return entrants

## Unit Tests

A handful of basic filter tests, utilising a dummy "event" class

In [3]:
class DummyEvent(Printable):
    def __init__(self, path, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path

        self.entrants = {}


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.logInfo('{} entrants loaded'.format(len(self.entrants)))

In [4]:
class TestFilter2019(unittest.TestCase):
    '''Class to test filters using 2019 data'''

    def testBoat(self):
        '''Test a simple boat filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Boat')

        self.assertEqual(len(entrants), 2)
        self.assertEqual(len(testFilter.cache), 1)


    def testKiteboard(self):
        '''Test a simple kiteboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Kiteboard')

        self.assertEqual(len(entrants), 10)
        self.assertEqual(len(testFilter.cache), 1)


    def testSailboard(self):
        '''Test a simple sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard')

        self.assertEqual(len(entrants), 56)
        self.assertEqual(len(testFilter.cache), 1)


    def testYouthFleetSailboard(self):
        '''Test a youth fleet sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,Status=Youth Fleet')

        self.assertEqual(len(entrants), 6)
        self.assertEqual(len(testFilter.cache), 2)


    def testFirstTimeSailboard(self):
        '''Test a first time sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,Status=First Timer')

        self.assertEqual(len(entrants), 5)
        self.assertEqual(len(testFilter.cache), 2)


    def testAmateurSailboard(self):
        '''Test a amateur sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,Status=Amateur')

        # Note that 5 of the 29 "amateur" are actually "first timer" and 6 are "youth fleet" 
        self.assertEqual(len(entrants), 29 - 5 - 6)
        self.assertEqual(len(testFilter.cache), 2)


    def testGoldFleetSailboard(self):
        '''Test a gold fleet sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,Status=Gold Fleet')

        self.assertEqual(len(entrants), 17)
        self.assertEqual(len(testFilter.cache), 2)


    def testProFleetSailboard(self):
        '''Test a pro fleet sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,Status=Pro Fleet')

        self.assertEqual(len(entrants), 10)
        self.assertEqual(len(testFilter.cache), 2)


    def testUkwaMember(self):
        '''Test a UKWA member filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,UKWA Member=Y')

        self.assertEqual(len(entrants), 32)
        self.assertEqual(len(testFilter.cache), 2)


    def testFemaleSailboard(self):
        '''Test a female sailboard filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Craft Type=Sailboard,Gender=F,Status<>Pro Fleet')

        # Note that 1 of the 4 women is "pro fleet" 
        self.assertEqual(len(entrants), 4 - 1)
        self.assertEqual(len(testFilter.cache), 3)


    def testTotals(self):
        '''Test that filters for boats, sailboards and kites add up to the expected total'''

        testFilter = Filter(event.entrants)
        boats = testFilter.getEntrants('Craft Type=Boat')
        kites = testFilter.getEntrants('Craft Type=Kiteboard')
        sailboards = testFilter.getEntrants('Craft Type=Sailboard')

        self.assertEqual(len(event.entrants), len(boats) + len(kites) + len(sailboards))
        self.assertEqual(len(testFilter.cache), 3)


    def testForcesGrouping(self):
        '''Test a forces filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Grouping=Forces - %')

        self.assertEqual(len(entrants), 5)
        self.assertEqual(len(testFilter.cache), 1)


    def testNonForcesGrouping(self):
        '''Test a non-forces filter'''

        testFilter = Filter(event.entrants)
        entrants = testFilter.getEntrants('Grouping<>Forces - %')

        # Note that 5 of the people with groupings are "forces" 
        self.assertEqual(len(entrants), 24 - 5)
        self.assertEqual(len(testFilter.cache), 1)

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [5]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    eventYear = '2019'
    eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
    event = DummyEvent(eventPath)
    event.loadEntrants()

    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.............
----------------------------------------------------------------------
Ran 13 tests in 0.036s

OK


## All Done!